In [0]:
import pandas as pd
import numpy as np
import os
import json
import re
import os
import datetime
from itertools import permutations 
from matplotlib import pyplot as plt
from google.colab import drive
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from scipy.sparse.linalg import svds
import networkx as nx

In [3]:
drive.mount('/content/drive')
folder = '/content/drive/My Drive/Hallo_Data/'
clean_folder = folder + 'clean_data/'
feat_folder = folder + 'features_added/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
users = pd.read_pickle(feat_folder+"users.pkl")
friends = pd.read_pickle(feat_folder+"friends.pkl")
users["friend_cnt"]

user_id
001GUmDfBjM1tx4alKHGTnYRUt62     0.0
001QkDMrFKORLwUgeGjJO7wXxnD2     1.0
0024bdecosW3PBRirMhwG5T0dPn2    11.0
002FanjKr3dJA92FOEtdcrWJPLY2     1.0
002GoCnNQ7PVFLXcMOYPFICzaJg1    15.0
                                ... 
zzx0NTpSicTD4acfqaB5k1HFhn33     3.0
zzx6deTfKhhAbAgD6cWp5XlnFwH2     0.0
zzzOh6WfLjQ2vacJ0yMj4bP6oZp2    11.0
zzzmMqmHbedtBb0Gp6fCHRlwYf83     0.0
zzzt4CujlpY8gbtIsrwAtBMxaGP2    31.0
Name: friend_cnt, Length: 267367, dtype: float64

In [5]:
#############
#process data
#############

augmented_users = users.copy()
COLS_TO_AUG = ["native_country", "native_lang"]

# COUNTRY_COMP_N = 4
# LANG_COMP_N = 4
PRINCIPAL_COMP_N = 4


for key in COLS_TO_AUG:
  #onehot incode values
  encoded_values = pd.get_dummies(users[key], drop_first=True)

  #run PCA on encoded values
  pca = PCA(n_components = PRINCIPAL_COMP_N)
  principal_components = pca.fit_transform(encoded_values)
  principal_df = pd.DataFrame(data = principal_components) \
                        .set_index(keys = users.index) \
                        .add_suffix("_pc_" + key)

  #add new principle componets
  augmented_users = pd.concat([augmented_users, principal_df], axis=1)


augmented_users = augmented_users.drop(COLS_TO_AUG, axis=1)
augmented_users

,last_status_update,points,streamer,follower_count,admin,coin_balance,bio,last_online,english_proficiency,banned,hop_on_rate,livestream_push_notify,suspended,push_notify,level,friend_cnt,followed_cnt,0_pc_native_country,1_pc_native_country,2_pc_native_country,3_pc_native_country,0_pc_native_lang,1_pc_native_lang,2_pc_native_lang,3_pc_native_lang
user_id,,,,,,,,,,,,,,,,,,,,,,,,,
001GUmDfBjM1tx4alKHGTnYRUt62,2019-07-28 12:30:51.844000+00:00,0.0,False,0,False,0,False,2019-07-28 12:30:51.844000+00:00,2,False,0.0,False,False,False,0.0,0.0,0.0,0.888334,0.011316,0.014992,0.015948,-0.076945,-0.347893,0.899941,0.025494
001QkDMrFKORLwUgeGjJO7wXxnD2,2019-07-31 02:18:36.962000+00:00,0.0,False,0,False,0,False,2019-07-31 02:18:36.962000+00:00,2,False,0.0,False,False,False,0.0,1.0,0.0,-0.159666,-0.083866,-0.175555,-0.502890,-0.072603,-0.313864,-0.246059,-0.667424
0024bdecosW3PBRirMhwG5T0dPn2,2019-06-11 15:52:35.621000+00:00,0.0,False,0,False,0,False,2019-06-11 15:52:35.621000+00:00,1,False,0.0,False,False,True,0.0,11.0,0.0,-0.091411,-0.013261,-0.019406,-0.023289,-0.072891,-0.316024,-0.270090,0.742052
002FanjKr3dJA92FOEtdcrWJPLY2,2019-07-02 10:31:59.306000+00:00,0.0,False,0,False,0,False,2019-07-02 10:31:59.306000+00:00,2,False,0.0,False,False,True,0.0,1.0,0.0,-0.093399,-0.013841,-0.020305,-0.024450,-0.054914,-0.201404,-0.029755,-0.004289
002GoCnNQ7PVFLXcMOYPFICzaJg1,2019-09-19 11:10:14.762000+00:00,0.0,False,0,False,0,False,2019-10-19 00:30:52.627000+00:00,2,False,0.0,False,False,True,0.0,15.0,0.0,-0.155083,-0.069272,-0.133164,-0.277109,0.756388,0.348829,0.010368,0.001268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzx0NTpSicTD4acfqaB5k1HFhn33,0,0.0,False,0,False,0,False,2019-08-20 17:35:45.400000+00:00,1,False,0.0,False,False,False,0.0,3.0,0.0,-0.139991,-0.041893,-0.069831,-0.103163,-0.066516,-0.270889,-0.080271,-0.014792
zzx6deTfKhhAbAgD6cWp5XlnFwH2,0,0.0,False,0,False,0,False,2019-09-29 08:53:49.353000+00:00,3,False,0.0,False,False,False,0.0,0.0,0.0,-0.094173,-0.014075,-0.020668,-0.024926,-0.055103,-0.202424,-0.030128,-0.004350
zzzOh6WfLjQ2vacJ0yMj4bP6oZp2,2019-10-05 15:20:40.699000+00:00,0.0,False,0,False,0,False,0,4,False,0.0,False,False,True,0.0,11.0,0.0,-0.159666,-0.083866,-0.175555,-0.502890,-0.072603,-0.313864,-0.246059,-0.667424


In [0]:
#friend connections
# A = np.array([[0, 1, 0, 1, 0, 0], 
#               [1, 0, 1, 0, 0, 0], 
#               [0, 1, 0, 1, 0, 0], 
#               [1, 0, 1, 0, 1, 0], 
#               [0, 0, 0, 1, 0, 1],
#               [0, 0, 0, 0, 1, 0]])

G = nx.DiGraph()
# G = nx.Graph()
G.add_nodes_from(users[users["friend_cnt"] > 0.0].index)
G.add_edges_from(friends[["user_id", "friend_id"]].values)
A = nx.adjacency_matrix(G)


print(A.shape)
m,n = A.shape
test_indicies = np.random.randint(0, m, size = n//3)

# test_data = A[:, test_indicies]
train_data = A.copy()
train_data[:, test_indicies] = 0

# def error(X, optim_H, optim_W):
#     return np.linalg.norm(X - optim_W @ optim_H, ord = 'fro')

#get sklearn apx
m = NMF(300)

#eigen people
W = m.fit_transform(train_data)
#coordinates of each person in the eigen people
H = m.components_

A_hat = np.round(np.dot(W, H), decimals = 3)
A_hat[A_hat > 0] = 1

import sklearn
sklearn.metrics.accuracy_score(A_hat, A)


# print("ERR SKL: ", error(A, H, W))

# print(np.round(np.dot(W, H), decimals = 3))

# print(H)

# since a person consists of who they know, 
# who they are closest to in a norm to is who they are most similar to
# thus they should be connected.
# so for each person, compare how similar they are to everyone else
# think about using the 1 norm

# now we can define a similarity metric on

(102121, 102121)


In [0]:
# ratings_list = [i.strip().split("::") for i in open('/users/nickbecker/Downloads/ml-1m/ratings.dat', 'r').readlines()]
# users_list = [i.strip().split("::") for i in open('/users/nickbecker/Downloads/ml-1m/users.dat', 'r').readlines()]
# movies_list = [i.strip().split("::") for i in open('/users/nickbecker/Downloads/ml-1m/movies.dat', 'r').readlines()]

# ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
# movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
# movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)


# R = R_df.as_matrix()
# user_ratings_mean = np.mean(R, axis = 1)
# R_demeaned = R - user_ratings_mean.reshape(-1, 1)


# U, sigma, Vt = svds(R_demeaned, k = 50)


# sigma = np.diag(sigma)


# all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
# preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)



# def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
#     # Get and sort the user's predictions
#     user_row_number = userID - 1 # UserID starts at 1, not 0
#     sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
#     # Get the user's data and merge in the movie information.
#     user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
#     user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
#                      sort_values(['Rating'], ascending=False)
#                  )

#     print 'User {0} has already rated {1} movies.'.format(userID, user_full.shape[0])
#     print 'Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations)
    
#     # Recommend the highest predicted rating movies that the user hasn't seen yet.
#     recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
#          merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
#                left_on = 'MovieID',
#                right_on = 'MovieID').
#          rename(columns = {user_row_number: 'Predictions'}).
#          sort_values('Predictions', ascending = False).
#                        iloc[:num_recommendations, :-1]
#                       )

#     return user_full, recommendations

# already_rated, predictions = recommend_movies(preds_df, 837, movies_df, ratings_df, 10)